# Ablation Study

## Imports

In [1]:
import pandas as pd
import os, sys, json, io, contextlib, warnings
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.nn as nn
# Point to your repo root if the notebook isn't already there
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), "../../"))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

# --- Quiet mode ---
warnings.filterwarnings("ignore", category=RuntimeWarning)
np.seterr(all="ignore")
@contextlib.contextmanager
def silence_output():
    buf = io.StringIO()
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        with contextlib.redirect_stdout(buf), contextlib.redirect_stderr(buf):
            yield

# --- Repro & device ---
np.random.seed(0)
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ---- Imports from your repo ----
from src.ClevelandMcGill.figure1 import Figure1
from src.ClevelandMcGill.figure12 import Figure12
from src.ClevelandMcGill.figure3 import Figure3
from src.ClevelandMcGill.figure4 import Figure4
from src.ClevelandMcGill.weber import Weber

from src.Datasets.testdataset import (
    TestDataset, test_normalization_data,
    test_pl_data_generation, test_pa_data_generation,
    test_bfr_data_generation, test_wb_data_generation, test_reg_data_generation,
)

from src.Models.cvt import CvTRegression
from src.Models.swin import SwinRegression
from src.Models.vit import ViTRegression
from src.Models.one_epoch_run import testingEpoch, testingEpochOne, testingEpoch_pre

# --- imports Pretrained models ---
from transformers import (
    CvtForImageClassification,
    SwinForImageClassification,
    ViTForImageClassification,
)


Using device: cuda


2025-10-15 16:23:25.885978: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-15 16:23:25.897343: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-15 16:23:25.900689: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-15 16:23:25.909859: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-15 16:23:26.507100: W tensorflow/compiler/tf2

## Task, Experiment types, Data Generation, Models

In [2]:
# ---- Tasks & experiment types ----
TASKS = ['Elementary Perceptual Task',
         'Position-Length',
         'Position-Angle',
         'Bar and Framed Rectangle',
         'Webers Law']

EL_DATATYPE_LIST = ['position_common_scale',
                    'position_non_aligned_scale',
                    'length',
                    'direction',
                    'angle',
                    'area',
                    'volume',
                    'curvature',
                    'shading']

PL_DATATYPE_LIST = ['data_to_type1',
                    'data_to_type2',
                    'data_to_type3',
                    'data_to_type4',
                    'data_to_type5']

PA_DATATYPE_LIST = ['data_to_barchart',
                    'data_to_piechart',
                    'data_to_piechart_aa']

BFR_DATATYPE_LIST = ['data_to_bars',
                     'data_to_framed_rectangles']

WB_DATATYPE_LIST = ['base10','base100','base1000']

DATATYPE_LIST = {TASKS[0]: EL_DATATYPE_LIST,
                 TASKS[1]: PL_DATATYPE_LIST,
                 TASKS[2]: PA_DATATYPE_LIST,
                 TASKS[3]: BFR_DATATYPE_LIST,
                 TASKS[4]: WB_DATATYPE_LIST}

# ---- Data generation  --
def data_gen(task_name, transform, experiment_type):
    if task_name == "Elementary Perceptual Task":
        DATATYPE = getattr(Figure1, experiment_type)
        X_test, y_test = test_reg_data_generation(DATATYPE, NOISE=True, test_target=100)

    elif task_name == "Position-Length":
        DATATYPE = getattr(Figure4, experiment_type)
        X_test, y_test = test_pl_data_generation(DATATYPE, NOISE=True, test_target=100)

    elif task_name == "Position-Angle":
        DATATYPE = getattr(Figure3, experiment_type)
        X_test, y_test = test_pa_data_generation(DATATYPE, NOISE=True, test_target=100)

    elif task_name == "Bar and Framed Rectangle":
        DATATYPE = getattr(Figure12, experiment_type)
        X_test, y_test = test_bfr_data_generation(DATATYPE, NOISE=True, test_target=100)

    elif task_name == "Webers Law":
        DATATYPE = getattr(Weber, experiment_type)
        X_test, y_test = test_wb_data_generation(DATATYPE, NOISE=True, test_target=100)

    else:
        raise ValueError(f"Unknown task_name: {task_name}")

    X_test = test_normalization_data(X_test)
    y_test = test_normalization_data(y_test)
    X_test -= 0.5
    return TestDataset(X_test, y_test, transform=transform, channels=True)

def data_loader(task_name, experiment_type, batch_size=64):
    transform = transforms.Compose([transforms.Resize((224, 224))])
    test_dataset = data_gen(task_name, transform, experiment_type)
    return DataLoader(test_dataset, batch_size, shuffle=True)


# ----- Data generation with diff image resolution ---

def data_gen_img_res(task_name, transform, experiment_type):
    if task_name == "Elementary Perceptual Task":
        DATATYPE = getattr(Figure1, experiment_type)
        X_test, y_test = test_reg_data_generation_res(DATATYPE, NOISE=True, test_target=100)

    elif task_name == "Position-Length":
        DATATYPE = getattr(Figure4, experiment_type)
        X_test, y_test = test_pl_data_generation_res(DATATYPE, NOISE=True, test_target=100)

    elif task_name == "Position-Angle":
        DATATYPE = getattr(Figure3, experiment_type)
        X_test, y_test = test_pa_data_generation_res(DATATYPE, NOISE=True, test_target=100)

    elif task_name == "Bar and Framed Rectangle":
        DATATYPE = getattr(Figure12, experiment_type)
        X_test, y_test = test_bfr_data_generation_res(DATATYPE, NOISE=True, test_target=100)

    elif task_name == "Webers Law":
        DATATYPE = getattr(Weber, experiment_type)
        X_test, y_test = test_wb_data_generation_res(DATATYPE, NOISE=True, test_target=100)

    else:
        raise ValueError(f"Unknown task_name: {task_name}")

    X_test = test_normalization_data(X_test)
    y_test = test_normalization_data(y_test)
    X_test -= 0.5
    return TestDataset(X_test, y_test, transform=transform, channels=True)

def data_loader_img_res(task_name, experiment_type, batch_size=64):
    transform = transforms.Compose([transforms.Resize((224, 224))])
    test_dataset = data_gen_img_res(task_name, transform, experiment_type)
    return DataLoader(test_dataset, batch_size, shuffle=True)

# ---- Models & helpers ----
MODELS = ["CVT", "SWIN", "VIT"]

def _num_outputs_for(task_name):
    if task_name == "Elementary Perceptual Task": return 1
    if task_name in ("Position-Length", "Position-Angle"): return 5
    if task_name == "Bar and Framed Rectangle": return 2
    if task_name == "Webers Law": return 1
    raise ValueError(f"Unknown task_name: {task_name}")


##  ========== LARGEDATA ==========

In [3]:
# ========== EVALUATE ALL (LARGEDATA) ==========
# Runs ALL tasks × ALL experiment types × {CVT, SWIN, VIT(vViT)} using largedata checkpoints.
# Saves per-run JSONs to results/largedata/ and a master CSV summary.

def _make_model(model_name, task_name):
    n_out = _num_outputs_for(task_name)
    if model_name == "CVT":  return CvTRegression(num_classes=n_out, channels=3)
    if model_name == "VIT":  return ViTRegression(num_classes=n_out, channels=3)
    if model_name == "SWIN": return SwinRegression(num_outputs=n_out, channels=3)
    raise ValueError(f"Unknown model: {model_name}")

# ---- Evaluate one model (largedata ckpts )----------
def test_model(model, model_name, task_name, exp_type, test_loader):
    model.to(device)
    ckpt = f"chkpt/chkpts_fromCluster/largedata/ld_{model_name.lower()}3channels_{exp_type}.pth"
    state = torch.load(ckpt, map_location=device)
    model.load_state_dict(state)

    with silence_output():
        if task_name in ("Position-Length", "Position-Angle", "Bar and Framed Rectangle"):
            m_error = testingEpoch(model, test_loader, device)
        else:
            m_error = testingEpochOne(model, test_loader, device)

    outdir = "results/largedata"
    os.makedirs(outdir, exist_ok=True)
    payload = {'Model': model_name, 'Task_name': task_name, 'Experiment_type': exp_type, 'MLAE': float(m_error)}
    with open(os.path.join(outdir, f"{model_name}_ld_{exp_type}.json"), "w") as f:
        json.dump(payload, f, indent=2)
    return float(m_error)

def evaluate_all_models(task_name, exp_type):
    test_loader = data_loader(task_name, exp_type)
    return {m: test_model(_make_model(m, task_name), m, task_name, exp_type, test_loader) for m in MODELS}

# ---- Run EVERYTHING ----
rows = []
for task in TASKS:
    print(f"\n=== TASK: {task} ===")
    for exp in DATATYPE_LIST[task]:
        print(f" -> Experiment: {exp}")
        results = evaluate_all_models(task, exp)
        for model_name, mlae in results.items():
            rows.append({"Task": task, "Experiment": exp, "Model": model_name, "MLAE": mlae})

df = pd.DataFrame(rows)

# Save master CSV
outdir = "results/largedata"
os.makedirs(outdir, exist_ok=True)
csv_all = os.path.join(outdir, "mlae_summary_all_tasks_ld.csv")
df.to_csv(csv_all, index=False)
print("\nSaved full MLAE table to:", csv_all)

# -------- Summaries --------
def _show(df_like, title=None):
    try:
        from IPython.display import display
        if title: print(title)
        display(df_like)
    except Exception:
        if title: print(title)
        print(df_like.to_string(index=False))

best_per_exp = (df.sort_values(["Task","Experiment","MLAE"])
                  .groupby(["Task","Experiment"], as_index=False)
                  .first())
_show(best_per_exp, "\nBest model per (Task, Experiment):")

avg_per_task_model = (df.groupby(["Task","Model"], as_index=False)["MLAE"]
                        .mean()
                        .sort_values(["Task","MLAE"]))
_show(avg_per_task_model, "\nAverage MLAE per Task × Model:")

print("\nDone (largedata).")



=== TASK: Elementary Perceptual Task ===
 -> Experiment: position_common_scale
Done 0.008746147155761719 seconds ( 100 iterations)
 -> Experiment: position_non_aligned_scale
Done 0.007737874984741211 seconds ( 100 iterations)
 -> Experiment: length
Done 0.007925748825073242 seconds ( 100 iterations)
 -> Experiment: direction
Done 0.010518550872802734 seconds ( 100 iterations)
 -> Experiment: angle
Done 0.010618209838867188 seconds ( 100 iterations)
 -> Experiment: area
Done 0.009296655654907227 seconds ( 100 iterations)
 -> Experiment: volume
Done 0.013582944869995117 seconds ( 100 iterations)
 -> Experiment: curvature
Done 0.013312101364135742 seconds ( 100 iterations)
 -> Experiment: shading
Done 0.06315326690673828 seconds ( 100 iterations)

=== TASK: Position-Length ===
 -> Experiment: data_to_type1
Done 0.027661800384521484 seconds ( 100 iterations)
 -> Experiment: data_to_type2
Done 0.023941516876220703 seconds ( 100 iterations)
 -> Experiment: data_to_type3
Done 0.0267174243927

,Task,Experiment,Model,MLAE
0,Bar and Framed Rectangle,data_to_bars,SWIN,4.741950
1,Bar and Framed Rectangle,data_to_framed_rectangles,SWIN,4.671744
2,Elementary Perceptual Task,angle,VIT,1.666913
3,Elementary Perceptual Task,area,SWIN,2.183496
4,Elementary Perceptual Task,curvature,SWIN,1.422289
5,Elementary Perceptual Task,direction,SWIN,0.267187
6,Elementary Perceptual Task,length,VIT,1.348657
7,Elementary Perceptual Task,position_common_scale,SWIN,2.171870
8,Elementary Perceptual Task,position_non_aligned_scale,VIT,1.634466
9,Elementary Perceptual Task,shading,SWIN,2.253807



Average MLAE per Task × Model:


,Task,Model,MLAE
1,Bar and Framed Rectangle,SWIN,4.706847
0,Bar and Framed Rectangle,CVT,4.800346
2,Bar and Framed Rectangle,VIT,5.144330
4,Elementary Perceptual Task,SWIN,2.172350
5,Elementary Perceptual Task,VIT,2.600044
3,Elementary Perceptual Task,CVT,4.005530
7,Position-Angle,SWIN,4.199882
6,Position-Angle,CVT,4.608053
8,Position-Angle,VIT,5.402470
10,Position-Length,SWIN,4.749202



Done (largedata).


##  ========== PRETRAINED ==========

In [4]:
# ========== EVALUATE ALL (PRETRAINED) ==========
# Builds  models (CVT/SWIN/ViT(vViT)), replaces classifier to match task outputs,
# loads your fine-tuned checkpoints, evaluates everything, and writes results.


def _make_model_pretrained(model_name: str, task_name: str):
    n_out = _num_outputs_for(task_name)

    if model_name == "CVT":
        model = CvtForImageClassification.from_pretrained("microsoft/cvt-13")
        in_dim = model.config.embed_dim[-1] if isinstance(model.config.embed_dim, (list, tuple)) else model.config.hidden_sizes[-1]
        model.classifier = nn.Linear(in_dim, n_out)
        return model

    if model_name == "SWIN":
        model = SwinForImageClassification.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
        in_dim = model.config.hidden_size
        model.classifier = nn.Linear(in_dim, n_out)
        return model

    if model_name == "VIT":
        model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224")
        in_dim = model.config.hidden_size
        model.classifier = nn.Linear(in_dim, n_out)
        return model

    raise ValueError(f"Unknown model: {model_name}")


def test_model(model_name, task_name, exp_type, test_loader):
    model = _make_model_pretrained(model_name, task_name)
    model.to(device)

    # Your fine-tuned checkpoints:
    ckpt = f"chkpt/chkpts_fromCluster/pretrained/pretrained_{model_name.lower()}_{exp_type}.pth"
    state = torch.load(ckpt, map_location=device)

    if isinstance(state, dict) and all(isinstance(k, str) for k in state.keys()):
        if "state_dict" in state and isinstance(state["state_dict"], dict):
            state = state["state_dict"]
        elif "model" in state and isinstance(state["model"], dict):
            state = state["model"]
        missing, unexpected = model.load_state_dict(state, strict=False)
        if missing or unexpected:
            pass
    else:
        model.load_state_dict(state, strict=False)

    with silence_output():
        if task_name in ("Position-Length", "Position-Angle", "Bar and Framed Rectangle"):
            m_error = testingEpoch_pre(model, test_loader, device)
        else:
            m_error = testingEpoch_pre(model, test_loader, device)

    outdir = "results/pretrained"
    os.makedirs(outdir, exist_ok=True)
    payload = {'Model': model_name, 'Task_name': task_name, 'Experiment_type': exp_type, 'MLAE': float(m_error)}
    with open(os.path.join(outdir, f"{model_name}_pretrained_{exp_type}.json"), "w") as f:
        json.dump(payload, f, indent=2)
    return float(m_error)

def evaluate_all_models(task_name, exp_type):
    test_loader = data_loader(task_name, exp_type)
    return {m: test_model(m, task_name, exp_type, test_loader) for m in MODELS}

# ---- Run EVERYTHING (pretrained) ----
import pandas as pd

rows = []
for task in TASKS:
    print(f"\n=== TASK: {task} ===")
    for exp in DATATYPE_LIST[task]:
        print(f" -> Experiment: {exp}")
        results = evaluate_all_models(task, exp)
        for model_name, mlae in results.items():
            rows.append({"Task": task, "Experiment": exp, "Model": model_name, "MLAE": mlae})

df = pd.DataFrame(rows)

# Save master CSV
outdir = "results/pretrained"
os.makedirs(outdir, exist_ok=True)
csv_all = os.path.join(outdir, "mlae_summary_all_tasks_pretrained.csv")
df.to_csv(csv_all, index=False)
print("\nSaved full MLAE table to:", csv_all)

# -------- Summaries --------
def _show(df_like, title=None):
    try:
        from IPython.display import display
        if title: print(title)
        display(df_like)
    except Exception:
        if title: print(title)
        print(df_like.to_string(index=False))

best_per_exp = (df.sort_values(["Task","Experiment","MLAE"])
                  .groupby(["Task","Experiment"], as_index=False)
                  .first())
_show(best_per_exp, "\nBest model per (Task, Experiment):")

avg_per_task_model = (df.groupby(["Task","Model"], as_index=False)["MLAE"]
                        .mean()
                        .sort_values(["Task","MLAE"]))
_show(avg_per_task_model, "\nAverage MLAE per Task × Model:")

print("\nDone (pretrained).")



=== TASK: Elementary Perceptual Task ===
 -> Experiment: position_common_scale
Done 0.011835098266601562 seconds ( 100 iterations)
 -> Experiment: position_non_aligned_scale
Done 0.0073394775390625 seconds ( 100 iterations)
 -> Experiment: length
Done 0.007215023040771484 seconds ( 100 iterations)
 -> Experiment: direction
Done 0.010700225830078125 seconds ( 100 iterations)
 -> Experiment: angle
Done 0.015317678451538086 seconds ( 100 iterations)
 -> Experiment: area
Done 0.009455680847167969 seconds ( 100 iterations)
 -> Experiment: volume
Done 0.019463777542114258 seconds ( 100 iterations)
 -> Experiment: curvature
Done 0.0174410343170166 seconds ( 100 iterations)
 -> Experiment: shading
Done 0.07379412651062012 seconds ( 100 iterations)

=== TASK: Position-Length ===
 -> Experiment: data_to_type1
Done 0.02655768394470215 seconds ( 100 iterations)
 -> Experiment: data_to_type2
Done 0.025231122970581055 seconds ( 100 iterations)
 -> Experiment: data_to_type3
Done 0.02677154541015625 

,Task,Experiment,Model,MLAE
0,Bar and Framed Rectangle,data_to_bars,SWIN,4.804684
1,Bar and Framed Rectangle,data_to_framed_rectangles,SWIN,4.736953
2,Elementary Perceptual Task,angle,SWIN,0.506287
3,Elementary Perceptual Task,area,CVT,0.942088
4,Elementary Perceptual Task,curvature,CVT,0.576370
5,Elementary Perceptual Task,direction,CVT,0.508800
6,Elementary Perceptual Task,length,CVT,0.251779
7,Elementary Perceptual Task,position_common_scale,VIT,1.186813
8,Elementary Perceptual Task,position_non_aligned_scale,SWIN,2.230774
9,Elementary Perceptual Task,shading,CVT,2.651150



Average MLAE per Task × Model:


,Task,Model,MLAE
1,Bar and Framed Rectangle,SWIN,4.770819
0,Bar and Framed Rectangle,CVT,4.797827
2,Bar and Framed Rectangle,VIT,4.955625
3,Elementary Perceptual Task,CVT,1.807189
4,Elementary Perceptual Task,SWIN,2.035161
5,Elementary Perceptual Task,VIT,2.143559
7,Position-Angle,SWIN,4.303269
6,Position-Angle,CVT,4.454274
8,Position-Angle,VIT,4.724473
10,Position-Length,SWIN,4.787086



Done (pretrained).


## ========== TINY patch8 ==========

In [5]:
# ========== EVALUATE ALL (TINY) ==========
# Runs ALL tasks × ALL experiment types × {tiny VIT(vViT)} using tiny ViT checkpoints.
# Saves per-run JSONs to results/tiny/ and CSV summary.


# Only VIT now
MODEL_NAME = "VIT"

def _make_model_tiny_vit(task_name: str):
    n_out = _num_outputs_for(task_name)
    return ViTRegression(
        num_classes=n_out,
        patch_size=(8, 8),
        dim=192,
        depth=12,
        heads=3,
        mlp_dim=768,
        channels=3
    )

# ---- Evaluate one model (tiny VIT ckpts) ----
def test_model_vit_only(task_name, exp_type, test_loader):
    model = _make_model_tiny_vit(task_name)
    model.to(device)

    ckpt = f"chkpt/chkpts_fromCluster/tiny/{MODEL_NAME.lower()}_tiny_patch8{exp_type}.pth"
    state = torch.load(ckpt, map_location=device)
    model.load_state_dict(state)

    with silence_output():
        if task_name in ("Position-Length", "Position-Angle", "Bar and Framed Rectangle"):
            m_error = testingEpoch(model, test_loader, device)
        else:
            m_error = testingEpochOne(model, test_loader, device)

    outdir = "results/tiny"
    os.makedirs(outdir, exist_ok=True)
    payload = {
        'Model': MODEL_NAME,
        'Task_name': task_name,
        'Experiment_type': exp_type,
        'MLAE': float(m_error)
    }
    with open(os.path.join(outdir, f"{MODEL_NAME}_tiny_patch8{exp_type}.json"), "w") as f:
        json.dump(payload, f, indent=2)

    return float(m_error)

def evaluate_all_models(task_name, exp_type):
    test_loader = data_loader(task_name, exp_type)
    mlae = test_model_vit_only(task_name, exp_type, test_loader)
    return {MODEL_NAME: mlae}

# ---- Run EVERYTHING (tiny VIT only) ----
import pandas as pd

rows = []
for task in TASKS:
    print(f"\n=== TASK: {task} ===")
    for exp in DATATYPE_LIST[task]:
        print(f" -> Experiment: {exp}")
        results = evaluate_all_models(task, exp)  # returns {"VIT": mlae}
        rows.append({
            "Task": task,
            "Experiment": exp,
            "Model": MODEL_NAME,
            "MLAE": results[MODEL_NAME]
        })

df = pd.DataFrame(rows)

# Save master CSV
outdir = "results/tiny"
os.makedirs(outdir, exist_ok=True)
csv_all = os.path.join(outdir, "mlae_summary_all_tasks_tiny_vit_only.csv")
df.to_csv(csv_all, index=False)
print("\nSaved full MLAE table to:", csv_all)

# -------- Summaries 
def _show(df_like, title=None):
    try:
        from IPython.display import display
        if title: print(title)
        display(df_like)
    except Exception:
        if title: print(title)
        print(df_like.to_string(index=False))

best_per_exp = (df.sort_values(["Task","Experiment","MLAE"])
                  .groupby(["Task","Experiment"], as_index=False)
                  .first())
_show(best_per_exp, "\nBest (VIT) MLAE per (Task, Experiment):")

avg_per_task_model = (df.groupby(["Task"], as_index=False)["MLAE"]
                        .mean()
                        .sort_values(["Task","MLAE"]))
avg_per_task_model.insert(1, "Model", MODEL_NAME)
_show(avg_per_task_model, "\nAverage MLAE per Task (VIT only):")

print("\nDone (tiny VIT only).")



=== TASK: Elementary Perceptual Task ===
 -> Experiment: position_common_scale
Done 0.012662172317504883 seconds ( 100 iterations)
 -> Experiment: position_non_aligned_scale
Done 0.007297515869140625 seconds ( 100 iterations)
 -> Experiment: length
Done 0.0072956085205078125 seconds ( 100 iterations)
 -> Experiment: direction
Done 0.00940561294555664 seconds ( 100 iterations)
 -> Experiment: angle
Done 0.010554790496826172 seconds ( 100 iterations)
 -> Experiment: area
Done 0.008894681930541992 seconds ( 100 iterations)
 -> Experiment: volume
Done 0.014128684997558594 seconds ( 100 iterations)
 -> Experiment: curvature
Done 0.0122833251953125 seconds ( 100 iterations)
 -> Experiment: shading
Done 0.0607149600982666 seconds ( 100 iterations)

=== TASK: Position-Length ===
 -> Experiment: data_to_type1
Done 0.027854204177856445 seconds ( 100 iterations)
 -> Experiment: data_to_type2
Done 0.025222063064575195 seconds ( 100 iterations)
 -> Experiment: data_to_type3
Done 0.0266463756561279

,Task,Experiment,Model,MLAE
0,Bar and Framed Rectangle,data_to_bars,VIT,5.387029
1,Bar and Framed Rectangle,data_to_framed_rectangles,VIT,5.409864
2,Elementary Perceptual Task,angle,VIT,4.629263
3,Elementary Perceptual Task,area,VIT,3.994583
4,Elementary Perceptual Task,curvature,VIT,2.274923
5,Elementary Perceptual Task,direction,VIT,3.396999
6,Elementary Perceptual Task,length,VIT,2.918858
7,Elementary Perceptual Task,position_common_scale,VIT,4.677745
8,Elementary Perceptual Task,position_non_aligned_scale,VIT,2.003672
9,Elementary Perceptual Task,shading,VIT,2.361739



Average MLAE per Task (VIT only):


,Task,Model,MLAE
0,Bar and Framed Rectangle,VIT,5.398447
1,Elementary Perceptual Task,VIT,3.281615
2,Position-Angle,VIT,5.508026
3,Position-Length,VIT,5.302922
4,Webers Law,VIT,7.448117



Done (tiny VIT only).


##  ========== IMAGE_RESOLUTION ============

In [6]:
# ========== EVALUATE ALL (Image Resolution is 224 during generation) ==========
# Runs ALL tasks × ALL experiment types × {CVT, SWIN, VIT(vViT)} using Image resolution with 224 during data generation and then using checkpoints.
# Saves per-run JSONs to results/largedata/ and a CSV summary.

#image_resolution to 224x224 during data/stimuli generation
from src.Datasets.testdataset_imgres import (
    TestDataset, test_normalization_data,
    test_pl_data_generation, test_pa_data_generation,
    test_bfr_data_generation, test_wb_data_generation, test_reg_data_generation,
)

def data_gen_img_res(task_name, transform, experiment_type):
    if task_name == "Elementary Perceptual Task":
        DATATYPE = getattr(Figure1, experiment_type)
        X_test, y_test = test_reg_data_generation(DATATYPE, NOISE=True, test_target=100)

    elif task_name == "Position-Length":
        DATATYPE = getattr(Figure4, experiment_type)
        X_test, y_test = test_pl_data_generation(DATATYPE, NOISE=True, test_target=100)

    elif task_name == "Position-Angle":
        DATATYPE = getattr(Figure3, experiment_type)
        X_test, y_test = test_pa_data_generation(DATATYPE, NOISE=True, test_target=100)

    elif task_name == "Bar and Framed Rectangle":
        DATATYPE = getattr(Figure12, experiment_type)
        X_test, y_test = test_bfr_data_generation(DATATYPE, NOISE=True, test_target=100)

    elif task_name == "Webers Law":
        DATATYPE = getattr(Weber, experiment_type)
        X_test, y_test = test_wb_data_generation(DATATYPE, NOISE=True, test_target=100)

    else:
        raise ValueError(f"Unknown task_name: {task_name}")

    X_test = test_normalization_data(X_test)
    y_test = test_normalization_data(y_test)
    X_test -= 0.5
    return TestDataset(X_test, y_test, transform=transform, channels=True)

def data_loader_img_res(task_name, experiment_type, batch_size=64):
    transform = transforms.Compose([transforms.Resize((224, 224))])
    test_dataset = data_gen_img_res(task_name, transform, experiment_type)
    return DataLoader(test_dataset, batch_size, shuffle=True)

    
def _make_model(model_name, task_name):
    n_out = _num_outputs_for(task_name)
    if model_name == "CVT":  return CvTRegression(num_classes=n_out, channels=3)
    if model_name == "VIT":  return ViTRegression(
        num_classes=n_out,
        patch_size=(16, 16),
        dim=192,
        depth=12,
        heads=3,
        mlp_dim=768,
        channels=3
    )
    if model_name == "SWIN": return SwinRegression(num_outputs=n_out, channels=3)
    raise ValueError(f"Unknown model: {model_name}")

# ---- Evaluate one model imgres ckpts----
def test_model(model, model_name, task_name, exp_type, test_loader):
    model.to(device)
    ckpt = f"chkpt/chkpts_fromCluster/img_res/{model_name.lower()}3_res_{exp_type}.pth"
    state = torch.load(ckpt, map_location=device)
    model.load_state_dict(state)

    with silence_output():
        if task_name in ("Position-Length", "Position-Angle", "Bar and Framed Rectangle"):
            m_error = testingEpoch(model, test_loader, device)
        else:
            m_error = testingEpochOne(model, test_loader, device)

    outdir = "results/imgres"
    os.makedirs(outdir, exist_ok=True)
    payload = {'Model': model_name, 'Task_name': task_name, 'Experiment_type': exp_type, 'MLAE': float(m_error)}
    with open(os.path.join(outdir, f"{model_name}_ld_{exp_type}.json"), "w") as f:
        json.dump(payload, f, indent=2)
    return float(m_error)

def evaluate_all_models(task_name, exp_type):
    test_loader = data_loader_img_res(task_name, exp_type)
    return {m: test_model(_make_model(m, task_name), m, task_name, exp_type, test_loader) for m in MODELS}

# ---- Run EVERYTHING ----
rows = []
for task in TASKS:
    print(f"\n=== TASK: {task} ===")
    for exp in DATATYPE_LIST[task]:
        print(f" -> Experiment: {exp}")
        results = evaluate_all_models(task, exp)
        for model_name, mlae in results.items():
            rows.append({"Task": task, "Experiment": exp, "Model": model_name, "MLAE": mlae})

df = pd.DataFrame(rows)

# Save master CSV
outdir = "results/imgres"
os.makedirs(outdir, exist_ok=True)
csv_all = os.path.join(outdir, "mlae_summary_all_tasks_imgres.csv")
df.to_csv(csv_all, index=False)
print("\nSaved full MLAE table to:", csv_all)

# -------- Summaries --------
def _show(df_like, title=None):
    try:
        from IPython.display import display
        if title: print(title)
        display(df_like)
    except Exception:
        if title: print(title)
        print(df_like.to_string(index=False))

best_per_exp = (df.sort_values(["Task","Experiment","MLAE"])
                  .groupby(["Task","Experiment"], as_index=False)
                  .first())
_show(best_per_exp, "\nBest model per (Task, Experiment):")

avg_per_task_model = (df.groupby(["Task","Model"], as_index=False)["MLAE"]
                        .mean()
                        .sort_values(["Task","MLAE"]))
_show(avg_per_task_model, "\nAverage MLAE per Task × Model:")

print("\nDone (image resolution).")



=== TASK: Elementary Perceptual Task ===
 -> Experiment: position_common_scale
 -> Experiment: position_non_aligned_scale
 -> Experiment: length
 -> Experiment: direction
 -> Experiment: angle
 -> Experiment: area
 -> Experiment: volume
 -> Experiment: curvature
 -> Experiment: shading

=== TASK: Position-Length ===
 -> Experiment: data_to_type1
 -> Experiment: data_to_type2
 -> Experiment: data_to_type3
 -> Experiment: data_to_type4
 -> Experiment: data_to_type5

=== TASK: Position-Angle ===
 -> Experiment: data_to_barchart
 -> Experiment: data_to_piechart
 -> Experiment: data_to_piechart_aa

=== TASK: Bar and Framed Rectangle ===
 -> Experiment: data_to_bars
 -> Experiment: data_to_framed_rectangles

=== TASK: Webers Law ===
 -> Experiment: base10
 -> Experiment: base100
 -> Experiment: base1000

Saved full MLAE table to: results/imgres/mlae_summary_all_tasks_imgres.csv

Best model per (Task, Experiment):


,Task,Experiment,Model,MLAE
0,Bar and Framed Rectangle,data_to_bars,SWIN,4.793733
1,Bar and Framed Rectangle,data_to_framed_rectangles,SWIN,4.669411
2,Elementary Perceptual Task,angle,CVT,4.524289
3,Elementary Perceptual Task,area,VIT,4.479436
4,Elementary Perceptual Task,curvature,SWIN,4.961404
5,Elementary Perceptual Task,direction,SWIN,3.880857
6,Elementary Perceptual Task,length,SWIN,3.562451
7,Elementary Perceptual Task,position_common_scale,VIT,4.260090
8,Elementary Perceptual Task,position_non_aligned_scale,CVT,4.727535
9,Elementary Perceptual Task,shading,SWIN,3.734190



Average MLAE per Task × Model:


,Task,Model,MLAE
1,Bar and Framed Rectangle,SWIN,4.731572
0,Bar and Framed Rectangle,CVT,4.880384
2,Bar and Framed Rectangle,VIT,5.536757
4,Elementary Perceptual Task,SWIN,4.522505
5,Elementary Perceptual Task,VIT,4.809127
3,Elementary Perceptual Task,CVT,4.838518
7,Position-Angle,SWIN,4.215089
6,Position-Angle,CVT,4.860096
8,Position-Angle,VIT,5.473652
10,Position-Length,SWIN,4.791236



Done (image resolution).
